In [ ]:
!pip install llama-index ragas

In [ ]:
# VectorStoreIndex is used for creating a searchable index
# SimpleDirectoryReader is used to read a directory with data
# ServiceContext is a utility container for LlamaIndex index and query classes
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
import os

In [ ]:
# OpenAI API Key
os.environ["OPENAI_API_KEY"] = "xxxxx"

In [ ]:
# Loading the data from the directory using SimpleDirectoryReader and building a VectorStoreIndex
documents = SimpleDirectoryReader("./data/").load_data()
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=ServiceContext.from_defaults(chunk_size=512)
)

# Initialising a query engine
query_engine = vector_index.as_query_engine()

In [ ]:
# passing our query to the query engine and generating response
response_vector = query_engine.query("Where is New York city?")

print(response_vector)

New York City is located at the southern tip of New York State.


In [ ]:
# RAGAS needs a set of questions and answers for evaluation
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,000",  # incorrect answer
    "Queens",  # incorrect answer
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

eval_answers = [[a] for a in eval_answers]

In [ ]:
# Importing RAGAS metrics
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from ragas.llama_index import evaluate

result = evaluate(query_engine, metrics, eval_questions, eval_answers)

evaluating with [faithfulness]


100%|██████████| 1/1 [00:19<00:00, 19.88s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:09<00:00,  9.14s/it]


evaluating with [context_precision]


100%|██████████| 1/1 [00:05<00:00,  5.47s/it]


evaluating with [context_recall]


100%|██████████| 1/1 [00:09<00:00,  9.63s/it]


evaluating with [harmfulness]


100%|██████████| 1/1 [00:01<00:00,  1.54s/it]


In [ ]:
# final scores
print(result)

{'faithfulness': 0.9000, 'answer_relevancy': 0.9819, 'context_precision': 0.3000, 'context_recall': 1.0000, 'harmfulness': 0.0000}
